In [ ]:
  #Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_set = "/content/drive/My Drive/Colab Notebooks/data/OLIDv1.0/olid-training-v1.0.tsv"
print("Reading Dataset...")
train_data = pd.read_csv(train_set, sep='\t', header=0)
print(train_data)

Reading Dataset...
          id  ... subtask_c
0      86426  ...       NaN
1      90194  ...       IND
2      16820  ...       NaN
3      62688  ...       NaN
4      43605  ...       NaN
...      ...  ...       ...
13235  95338  ...       IND
13236  67210  ...       NaN
13237  82921  ...       OTH
13238  27429  ...       NaN
13239  46552  ...       NaN

[13240 rows x 5 columns]


In [ ]:
tweets = train_data[["tweet"]]
subtask_a_labels = train_data[["subtask_a"]]
subtask_b_labels = train_data.query("subtask_a == 'OFF'")[["subtask_b"]]
subtask_c_labels = train_data.query("subtask_b == 'TIN'")[["subtask_c"]]

clean_tweets = copy.deepcopy(tweets)

In [ ]:
##PREPROCESSING##

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def take_data_to_shower(tweet):
    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)


def tokenize(tweet):
    lower_tweet = tweet.lower()
    return word_tokenize(lower_tweet)


def remove_stop_words(tokens):
    clean_tokens = []
    stopWords = set(stopwords.words('english'))
    for token in tokens:
        if token not in stopWords:
            if token.replace(' ', '') != '':
                if len(token) > 1:
                    clean_tokens.append(token)
    return clean_tokens


def stem_and_lem(tokens):
    clean_tokens = []
    for token in tokens:
        token = wordnet_lemmatizer.lemmatize(token)
        token = lancaster_stemmer.stem(token)
        if len(token) > 1:
            clean_tokens.append(token)
    return clean_tokens

def replace_abbreviations(tweet):
    texts = []
    for item in text:
        item = item.lower().replace("it's", "it is").replace("i'm", "i am").replace("he's", "he is").replace("she's", "she is")\
            .replace("we're", "we are").replace("they're", "they are").replace("you're", "you are").replace("that's", "that is")\
            .replace("this's", "this is").replace("can't", "can not").replace("don't", "do not").replace("doesn't", "does not")\
            .replace("we've", "we have").replace("i've", " i have").replace("isn't", "is not").replace("won't", "will not")\
            .replace("hasn't", "has not").replace("wasn't", "was not").replace("weren't", "were not").replace("let's", "let us")\
            .replace("didn't", "did not").replace("hadn't", "had not").replace("waht's", "what is").replace("couldn't", "could not")\
            .replace("you'll", "you will").replace("you've", "you have")
        item = item.replace("'s", "")
        texts.append(item)
    return tweet

In [ ]:
tqdm.pandas(desc="Cleaning Data Phase I...")
clean_tweets['tweet'] = tweets['tweet'].progress_apply(take_data_to_shower)

tqdm.pandas(desc="Tokenizing Data...")
clean_tweets['tokens'] = clean_tweets['tweet'].progress_apply(tokenize)

tqdm.pandas(desc="Cleaning Data Phase II...")
clean_tweets['tokens'] = clean_tweets['tokens'].progress_apply(remove_stop_words)

tqdm.pandas(desc="Stemming And Lemmatizing")
clean_tweets['tokens'] = clean_tweets['tokens'].progress_apply(stem_and_lem)

text_vector = clean_tweets['tokens'].tolist()
print(text_vector)

Stemming And Lemmatizing: 100%|██████████| 13240/13240 [00:04<00:00, 3276.81it/s]

[['ask', 'nat', 'am', 'tak'], ['go', 'hom', 'drunk', 'mag', 'trump'], ['amazon', 'investig', 'chines', 'employ', 'sel', 'intern', 'dat', 'third', 'party', 'sel', 'look', 'edg', 'competit', 'marketplac', 'amazon', 'mag', 'kag', 'chin', 'tcot'], ['someon', 'shouldtak', 'piec', 'shit', 'volcano'], ['obam', 'want', 'lib', 'amp', 'illeg', 'mov', 'red', 'stat'], ['lib', 'kookoo'], ['oh', 'no', 'tough', 'shit'], ['lit', 'talk', 'lol', 'mass', 'shoot', 'lik', 'set', 'up', 'propagand', 'us', 'divid', 'maj', 'issu', 'lik', 'gun', 'control', 'ter'], ['buy', 'icecream'], ['canad', 'nee', 'anoth', 'cuck', 'already', 'enough', 'looneyleft', 'lib', 'king', 'gre', 'country', 'qproofs', 'trudeaumustgo'], ['fault', 'support', 'gun', 'control'], ['diff', 'kavanaugh', 'on', 'men', 'admit', 'grop', 'year', 'old', 'girl', 'year', 'ago', 'going', 'confirm', 'scj', 'demsarefraud', 'demsaredon', 'walkawaydemocr', 'redwav', 'voteredsaveameric', 'trumptrain', 'mag'], ['lying', 'corrupt', 'trait', 'nobody', 'want

In [ ]:
##EMBEDDING##

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfid(text_vector):
    vectorizer = TfidfVectorizer()
    untokenized_data =[' '.join(tweet) for tweet in tqdm(text_vector, "Vectorizing...")]
    vectorizer = vectorizer.fit(untokenized_data)
    vectors = vectorizer.transform(untokenized_data).toarray()
    return vectors
  
def get_vectors(vectors, labels, keyword):
    if len(vectors) != len(labels):
        print("Unmatching sizes!")
        return
    result = list()
    for vector, label in zip(vectors, labels):
        if label == keyword:
            result.append(vector)
    return result

In [ ]:
vectors_a = tfid(text_vector) # Numerical Vectors A
labels_a = subtask_a_labels['subtask_a'].values.tolist() # Subtask A Labels

vectors_b = get_vectors(vectors_a, labels_a, "OFF") # Numerical Vectors B
labels_b = subtask_b_labels['subtask_b'].values.tolist() # Subtask B Labels

vectors_c = get_vectors(vectors_b, labels_b, "TIN") # Numerical Vectors C
labels_c = subtask_c_labels['subtask_c'].values.tolist() # Subtask C Labels

Vectorizing...: 100%|██████████| 13240/13240 [00:00<00:00, 1118165.77it/s]


In [ ]:
##CLASSIFING##

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, matthews_corrcoef
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

def classify(vectors, labels, type="DT"):
    # Random Splitting With Ratio 3 : 1
    train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, test_size=0.25)

    # Initialize Model
    classifier = None
    if(type=="MNB"):
        classifier = MultinomialNB(alpha=0.7)
        classifier.fit(train_vectors, train_labels)
    elif(type=="KNN"):
        classifier = KNeighborsClassifier(n_jobs=4)
        params = {'n_neighbors': [3,5,7,9], 'weights':['uniform', 'distance']}
        classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
        classifier.fit(train_vectors, train_labels)
        classifier = classifier.best_estimator_
    elif(type=="SVM"):
        classifier = SVC()
        classifier = GridSearchCV(classifier, {'C':[0.001, 0.01, 0.1, 1, 10]}, cv=3, n_jobs=4)
        classifier.fit(train_vectors, train_labels)
        classifier = classifier.best_estimator_
        w = classifier.coef_  
        b = classifier.intercept_
    elif(type=="DT"):
        classifier = DecisionTreeClassifier(max_depth=800, min_samples_split=5)
        params = {'criterion':['gini','entropy']}
        classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
        classifier.fit(train_vectors, train_labels)
        classifier = classifier.best_estimator_
    elif(type=="RF"):
        classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
        params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
        classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
        classifier.fit(train_vectors, train_labels)
        classifier = classifier.best_estimator_
    elif(type=="LR"):
        classifier = LogisticRegression(multi_class='auto', solver='newton-cg',)
        classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
        classifier.fit(train_vectors, train_labels)
        classifier = classifier.best_estimator_
    else:
        print("Wrong Classifier Type!")
        return

    accuracy = accuracy_score(train_labels, classifier.predict(train_vectors))
    print("Training Accuracy:", accuracy)
    test_predictions = classifier.predict(test_vectors)
    accuracy = accuracy_score(test_labels, test_predictions)
    precision = precision_score(test_labels, test_predictions, average=None)
    recall = recall_score(test_labels, test_predictions, average=None)
    f1 =  f1_score(test_labels, test_predictions, average=None)
    f1_macro =  f1_score(test_labels, test_predictions, average='macro')
    f1_micro =  f1_score(test_labels, test_predictions, average='micro')
    f1_weight =  f1_score(test_labels, test_predictions, average='weighted')
    MCC = matthews_corrcoef(test_labels, test_predictions, sample_weight=None)
    print("Test Accuracy:", accuracy)
    print("Test Precision:", precision)
    print("Test Recall:", recall)
    print( "f1_score:", f1)
    print( "f1_macro:", f1_macro)
    print( "f1_micro:", f1_micro)
    print( "f1_weight:", f1_weight)
    print( "MCC:", MCC)
    print("Confusion Matrix:",)
    print(confusion_matrix(test_labels, test_predictions))

ROC and Confusion matrix

In [ ]:
'''
from sklearn.metrics import roc_curve, auc
import matplotlib as mpl  
import matplotlib.pyplot as plt

def plot_roc(labels, predict_prob):
    false_positive_rate,true_positive_rate,thresholds=roc_curve(labels, predict_prob)
    roc_auc=auc(false_positive_rate, true_positive_rate)
    plt.title('ROC')
    plt.plot(false_positive_rate, true_positive_rate,'b',label='AUC = %0.4f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.ylabel('TPR')
    plt.xlabel('FPR')

def confusion_matrix(test_labels, test_predictions):
    plt.imshow(confusion, cmap=plt.cm.Blues)
    indices = range(len(confusion_matrix))
    plt.xticks(indices, classes)
    plt.yticks(indices, classes)
    plt.colorbar()
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    for first_index in range(len(confusion)):
        for second_index in range(len(confusion[first_index])):
            plt.text(first_index, second_index, confusion[first_index][second_index])
'''

"\nfrom sklearn.metrics import roc_curve, auc\nimport matplotlib as mpl  \nimport matplotlib.pyplot as plt\n\ndef plot_roc(labels, predict_prob):\n    false_positive_rate,true_positive_rate,thresholds=roc_curve(labels, predict_prob)\n    roc_auc=auc(false_positive_rate, true_positive_rate)\n    plt.title('ROC')\n    plt.plot(false_positive_rate, true_positive_rate,'b',label='AUC = %0.4f'% roc_auc)\n    plt.legend(loc='lower right')\n    plt.plot([0,1],[0,1],'r--')\n    plt.ylabel('TPR')\n    plt.xlabel('FPR')\n\ndef confusion_matrix(test_labels, test_predictions):\n    plt.imshow(confusion, cmap=plt.cm.Blues)\n    indices = range(len(confusion_matrix))\n    plt.xticks(indices, classes)\n    plt.yticks(indices, classes)\n    plt.colorbar()\n    plt.xlabel('Predicted label')\n    plt.ylabel('True label')\n    for first_index in range(len(confusion)):\n        for second_index in range(len(confusion[first_index])):\n            plt.text(first_index, second_index, confusion[first_index][se

Decision Tree

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "DT") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...
Training Accuracy: 0.980563947633434
Test Accuracy: 0.7271903323262839
Test Precision: [0.78892281 0.58800393]
Test Recall: [0.81193896 0.55268022]
f1_score: [0.80026543 0.56979514]
f1_macro: 0.685030284272885
f1_micro: 0.7271903323262839
f1_weight: 0.724927406543071
MCC: 0.37072188992370875
Confusion Matrix:
[[1809  419]
 [ 484  598]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "DT") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...
Training Accuracy: 0.9806060606060606
Test Accuracy: 0.8472727272727273
Test Precision: [0.90239044 0.27083333]
Test Recall: [0.92827869 0.20967742]
f1_score: [0.91515152 0.23636364]
f1_macro: 0.5757575757575758
f1_micro: 0.8472727272727273
f1_weight: 0.838633608815427
MCC: 0.1545874423082925
Confusion Matrix:
[[906  70]
 [ 98  26]]


In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "DT") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...
Training Accuracy: 0.9594083247334021
Test Accuracy: 0.6078431372549019
Test Precision: [0.45126354 0.7274168  0.08196721]
Test Recall: [0.49212598 0.74271845 0.05154639]
f1_score: [0.47080979 0.73498799 0.06329114]
f1_macro: 0.4230296408255037
f1_micro: 0.6078431372549019
f1_weight: 0.598501038133206
MCC: 0.22160601814513484
Confusion Matrix:
[[125 108  21]
 [124 459  35]
 [ 28  64   5]]


MNB

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "MNB") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...
Training Accuracy: 0.7961732124874119
Test Accuracy: 0.7235649546827795
Test Precision: [0.71447587 0.81311475]
Test Recall: [0.97413793 0.22423146]
f1_score: [0.82434248 0.35152374]
f1_macro: 0.5879331130944793
f1_micro: 0.7235649546827795
f1_weight: 0.6663553153398472
MCC: 0.3235086305194187
Confusion Matrix:
[[2147   57]
 [ 858  248]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "MNB") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...
Training Accuracy: 0.8815151515151515
Test Accuracy: 0.8790909090909091
Test Precision: [0.87909091 0.        ]
Test Recall: [1. 0.]
f1_score: [0.93565554 0.        ]
f1_macro: 0.4678277697145622
f1_micro: 0.8790909090909091
f1_weight: 0.8225262787526939
MCC: 0.0
Confusion Matrix:
[[967   0]
 [133   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "MNB") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...
Training Accuracy: 0.6762985896112831
Test Accuracy: 0.6367389060887513
Test Precision: [0.6        0.63751317 0.        ]
Test Recall: [0.04511278 0.99180328 0.        ]
f1_score: [0.08391608 0.77613855 0.        ]
f1_macro: 0.28668487808962473
f1_micro: 0.6367389060887513
f1_weight: 0.5116266192331067
MCC: 0.10044197790415253
Confusion Matrix:
[[ 12 254   0]
 [  5 605   0]
 [  3  90   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KNeighborsClassifier

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "KNN") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.7395770392749245
Test Accuracy: 0.6812688821752266
Test Precision: [0.68071721 0.69465649]
Test Recall: [0.98185118 0.08227848]
f1_score: [0.80401263 0.14713015]
f1_macro: 0.47557139297927503
f1_micro: 0.6812688821752266
f1_weight: 0.5845225956503579
MCC: 0.15515343278444443
Confusion Matrix:
[[2164   40]
 [1015   91]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "KNN") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...
Training Accuracy: 0.889090909090909
Test Accuracy: 0.8690909090909091
Test Precision: [0.87940631 0.36363636]
Test Recall: [0.98544699 0.05797101]
f1_score: [0.92941176 0.1       ]
f1_macro: 0.5147058823529412
f1_micro: 0.8690909090909091
f1_weight: 0.8253582887700536
MCC: 0.10272500523969334
Confusion Matrix:
[[948  14]
 [130   8]]


In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "KNN") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.6697626418988648
Test Accuracy: 0.6398348813209495
Test Precision: [0.54166667 0.64804469 0.5       ]
Test Recall: [0.14338235 0.96345515 0.01052632]
f1_score: [0.22674419 0.7748831  0.02061856]
f1_macro: 0.3407486140933136
f1_micro: 0.6398348813209495
f1_weight: 0.5470720406705395
MCC: 0.16492951795767846
Confusion Matrix:
[[ 39 233   0]
 [ 21 580   1]
 [ 12  82   1]]


SVM

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "SVM") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.9966767371601208
Test Accuracy: 0.7616314199395771
Test Precision: [0.77222008 0.72292546]
Test Recall: [0.91061706 0.46473779]
f1_score: [0.83572767 0.56576775]
f1_macro: 0.7007477096214976
f1_micro: 0.7616314199395771
f1_weight: 0.74552353944685
MCC: 0.4311093624652722
Confusion Matrix:
[[2007  197]
 [ 592  514]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "SVM") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...


AttributeError: ignored

In [ ]:
w = classify.coef_  
b = classify.intercept_

AttributeError: ignored

In [ ]:
   # plot
    # 绘制分类平面
    ax = plt.subplot(111, projection='3d')
    x = np.arange(0,1,0.01)
    y = np.arange(0,1,0.11)
    x, y = np.meshgrid(x, y)
    z = (w[0,0]*x + w[0,1]*y + b) / (-w[0,2])
    surf = ax.plot_surface(x, y, z, rstride=1, cstride=1)

    # 绘制三维散点图
    x_array = np.array(x_train, dtype=float)
    y_array = np.array(y_train, dtype=int)
    pos = x_array[np.where(y_array==1)]
    neg = x_array[np.where(y_array==-1)]
    ax.scatter(pos[:,0], pos[:,1], pos[:,2], c='r', label='pos')
    ax.scatter(neg[:,0], neg[:,1], neg[:,2], c='b', label='neg')

    # 绘制支持向量

    X = np.array(x_train,dtype=float)
    for i in range(len(sv_idx)):
        ax.scatter(X[sv_idx[i],0], X[sv_idx[i],1], X[sv_idx[i],2],s=50,
                   c='',marker='o', edgecolors='g')

    # 绘制 bad case
    # x_test = np.array(x_test,dtype=float)
    # for i in range(len(bad_idx)):
    #     j = bad_idx[i]
    #     ax.scatter(x_test[j,0], x_test[j,1], x_test[j,2],s=60,
    #                c='',marker='o', edgecolors='g')

    ax.set_zlabel('Z')    # 坐标轴
    ax.set_ylabel('Y')
    ax.set_xlabel('X')
    ax.set_zlim([0, 1])
    plt.legend(loc='upper left')

    ax.view_init(35,300)
    plt.show()

In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "SVM") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...
Training Accuracy: 0.998624011007912
Test Accuracy: 0.7048503611971104
Test Precision: [0.56701031 0.74445893 0.25      ]
Test Recall: [0.43307087 0.90779014 0.02325581]
f1_score: [0.49107143 0.81805158 0.04255319]
f1_macro: 0.4505587319973408
f1_micro: 0.7048503611971104
f1_weight: 0.6635151275397038
MCC: 0.33693814355891705
Confusion Matrix:
[[110 140   4]
 [ 56 571   2]
 [ 28  56   2]]


Random Forest

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "RF") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.9921450151057402
Test Accuracy: 0.7755287009063444
Test Precision: [0.77264256 0.78787879]
Test Recall: [0.93970988 0.44746377]
f1_score: [0.84802618 0.57076834]
f1_macro: 0.7093972616109993
f1_micro: 0.7755287009063443
f1_weight: 0.7555510590166169
MCC: 0.4658530862480204
Confusion Matrix:
[[2073  133]
 [ 610  494]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "RF") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.9818181818181818
Test Accuracy: 0.8809090909090909
Test Precision: [0.88940092 0.26666667]
Test Recall: [0.98872951 0.03225806]
f1_score: [0.93643862 0.05755396]
f1_macro: 0.49699628943133706
f1_micro: 0.8809090909090909
f1_weight: 0.837364350679044
MCC: 0.05723180809745427
Confusion Matrix:
[[965  11]
 [120   4]]


In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "RF") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...
Training Accuracy: 0.9955280357757138
Test Accuracy: 0.6862745098039216
Test Precision: [0.65269461 0.69413233 0.        ]
Test Recall: [0.38790036 0.93132328 0.        ]
f1_score: [0.48660714 0.79542203 0.        ]
f1_macro: 0.4273430581102255
f1_micro: 0.6862745098039216
f1_weight: 0.6311698244918026
MCC: 0.3261919057284274
Confusion Matrix:
[[109 172   0]
 [ 40 556   1]
 [ 18  73   0]]


LogisticRegression

In [ ]:
print("\nBuilding Model Subtask A...")
classify(vectors_a[:], labels_a[:], "LR") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask A...


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Training Accuracy: 0.8123867069486405
Test Accuracy: 0.7516616314199396
Test Precision: [0.74089936 0.81102362]
Test Recall: [0.9558011  0.36203866]
f1_score: [0.83474065 0.50060753]
f1_macro: 0.6676740924007758
f1_micro: 0.7516616314199397
f1_weight: 0.7198634646037899
MCC: 0.418835376395843
Confusion Matrix:
[[2076   96]
 [ 726  412]]


In [ ]:
print("\nBuilding Model Subtask B...")
classify(vectors_b[:], labels_b[:], "LR") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask B...
Training Accuracy: 0.8842424242424243
Test Accuracy: 0.8709090909090909
Test Precision: [0.87090909 0.        ]
Test Recall: [1. 0.]
f1_score: [0.93100097 0.        ]
f1_macro: 0.4655004859086492
f1_micro: 0.8709090909090909
f1_weight: 0.8108172100008835
MCC: 0.0
Confusion Matrix:
[[958   0]
 [142   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
print("\nBuilding Model Subtask C...")
classify(vectors_c[:], labels_c[:], "LR") # {MNB, KNN, SVM, DT, RF, LR}


Building Model Subtask C...
Training Accuracy: 0.7936016511867905
Test Accuracy: 0.6780185758513931
Test Precision: [0.62068966 0.69104666 0.5       ]
Test Recall: [0.375      0.92255892 0.01149425]
f1_score: [0.46753247 0.79019466 0.02247191]
f1_macro: 0.4267330141296264
f1_micro: 0.6780185758513931
f1_weight: 0.6253663959619121
MCC: 0.30961967134534246
Confusion Matrix:
[[108 180   0]
 [ 45 548   1]
 [ 21  65   1]]
